<h1 style="color:#2E86C1; font-family: 'Helvetica';">d*liteOptimized.py</h2>

<p style="font-size: 16px; line-height: 1.5; color:#333;">
Implementation of the <b>D* Lite</b> dynamic path replaning algorithm.
</p>

In [183]:
import os
import math
import sys
import heapq
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Dict, List, Tuple
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(1) Data Preprocessing</h2>

In [184]:
@dataclass
class DStarLiteState:
    S: set
    succ: dict
    pred: dict
    cost_forecast: dict
    cost_nominal: dict
    node_positions: dict
    g: dict = field(default_factory=lambda: defaultdict(lambda: float('inf')))
    rhs: dict = field(default_factory=lambda: defaultdict(lambda: float('inf')))
    U: list = field(default_factory=list)
    k_m: float = 0.0
    s_start: int = None
    s_goal: int = None
    R_SENSE: int = field(default=1)
    robust_initialized: bool = False

In [185]:
def load_graph_data(nodes_csv: str, edges_csv: str):
    nodes = pd.read_csv(nodes_csv)
    edges = pd.read_csv(edges_csv)
    id_col = "id" if "id" in nodes.columns else "node_id"

    S = set(nodes[id_col].astype(int).tolist())
    node_positions = {int(row[id_col]): (row["x"], row["y"]) for _, row in nodes.iterrows()}
    cost_forecast = {(int(row.u), int(row.v)): float(row.cost) for _, row in edges.iterrows()}
    cost_nominal = {(int(row.u), int(row.v)): float(row.nominal) for _, row in edges.iterrows()}

    succ = defaultdict(list)
    pred = defaultdict(list)
    for _, row in edges.iterrows():
        u, v = int(row.u), int(row.v)
        succ[u].append(v)
        pred[v].append(u)

    return S, succ, pred, cost_forecast, cost_nominal, node_positions

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(2) Helper Functions</h2>


In [186]:
def c(u, v, cost_dict):
    return cost_dict.get((u, v), float('inf'))

In [187]:
def h(a, b, node_positions):
    ax, ay = node_positions[a]
    bx, by = node_positions[b]
    return abs(ax - bx) + abs(ay - by)

In [188]:
def KeyLessThan(k1, k2):
    eps = 1e-12
    if k1[0] < k2[0] - eps: return True
    if k1[0] > k2[0] + eps: return False
    return k1[1] < k2[1] - eps

In [189]:
def load_robust_path(path_file: str) -> List[int]:
    if not os.path.exists(path_file):
        raise FileNotFoundError(f"Robust path file not found: {path_file}")
    df = pd.read_csv(path_file)
    col = df.columns[0]
    path_nodes = df[col].astype(int).tolist()
    print(f"Loaded robust path with {len(path_nodes)} nodes from {path_file}")
    return path_nodes

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(3) Priority Queue Operations</h2>

In [190]:
def U_Remove(U, s):
    U[:] = [(key, node) for key, node in U if node != s]
    heapq.heapify(U)

In [191]:
def U_Insert(U, u, k):
    heapq.heappush(U, (k, u))

In [192]:
def U_Update(U, s, k):
    U_Remove(U, s)
    heapq.heappush(U, (k, s))

In [193]:
def U_TopKey(U):
    return U[0][0] if U else (float('inf'), float('inf'))

In [194]:
def U_Top(U):
    return U[0][1] if U else None

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(3) Data Postprocessing</h2>

In [195]:
def build_dstar_breakdown(g, rhs, U, path_nodes, succ, cost_forecast, cost_nominal):
    rows, total_forecast, total_nominal = [], 0.0, 0.0
    for idx in range(len(path_nodes) - 1):
        u, v = path_nodes[idx], path_nodes[idx + 1]
        forecast_cost = cost_forecast.get((u, v), float('inf'))
        actual_cost = cost_nominal.get((u, v), float('inf'))
        deviation = actual_cost - forecast_cost
        total_forecast += forecast_cost
        total_nominal += actual_cost
        rows.append({
            "order": idx + 1,
            "u": u, "v": v,
            "forecast_cost": forecast_cost,
            "actual_cost": actual_cost,
            "deviation": deviation,
            "g_value": g[u],
            "rhs_value": rhs[u],
            "in_queue": any(node == u for _, node in U)
        })
    rows.append({
        "order": "TOTAL_FORECAST_COST",
        "forecast_cost": total_forecast,
        "actual_cost": "",
        "deviation": "",
        "g_value": "",
        "rhs_value": "",
        "in_queue": ""
    })
    return pd.DataFrame(rows), total_forecast, total_nominal

In [196]:
def save_dstar_results(out_dir, breakdown_df, path_nodes):
    os.makedirs(out_dir, exist_ok=True)
    breakdown_csv = os.path.join(out_dir, "dstar_solution_breakdown.csv")
    path_csv = os.path.join(out_dir, "dstar_path_nodes.csv")
    breakdown_df.to_csv(breakdown_csv, index=False)
    pd.DataFrame({"path_node_id": path_nodes}).to_csv(path_csv, index=False)
    print(f"Saved D* Lite breakdown → {breakdown_csv}")
    print(f"Saved D* Lite path nodes → {path_csv}")
    return breakdown_csv, path_csv

In [197]:
def plot_dstar_path_overlay(nodes_csv, edges_csv, cost_forecast, path_nodes, output_dir, total_forecast=None, total_nominal=None, robust_path=None):
    forecast_npy = edges_csv.replace("edges.csv", "actual.npy")
    cost_field = np.load(forecast_npy).T

    nodes = pd.read_csv(nodes_csv)
    id_to_idx = {int(row.id if "id" in row else row.node_id): i for i, row in nodes.iterrows()}
    coords = nodes[["x", "y"]].to_numpy()
    path_xy = np.array([coords[id_to_idx[n]] for n in path_nodes])

    plt.figure(figsize=(8, 8))
    im = plt.imshow(cost_field, origin="lower", cmap="viridis")
    plt.colorbar(im, label="Forecast Cost (normalized)")

    # --- D* Lite path ---
    plt.plot(path_xy[:, 0], path_xy[:, 1], color="red", linewidth=2, label="D* Lite Path")
    plt.scatter(path_xy[0, 0], path_xy[0, 1], color="orange", s=60, label="Start")
    plt.scatter(path_xy[-1, 0], path_xy[-1, 1], color="cyan", s=60, label="Goal")

    # --- Robust overlay (blue dashed) ---
    if robust_path is not None:
        robust_xy = np.array([coords[id_to_idx[n]] for n in robust_path])
        plt.plot(
            robust_xy[:, 0], robust_xy[:, 1],
            color="blue", linestyle="--", linewidth=2.0, label="Robust Path"
        )

    plt.legend()
    title = "D* Lite Path Overlay (forecast background, actual discovered)"
    if total_forecast is not None and total_nominal is not None:
        title += f"\nTotal Forecast Cost: {total_forecast:.2f} | Total Nominal Cost: {total_nominal:.2f}"
    plt.title(title)
    plt.tight_layout()
    out_png = os.path.join(output_dir, "dstar_path_overlay.png")
    plt.savefig(out_png)
    plt.close()
    print(f"Saved D* Lite visualization → {out_png}")
    return out_png

<h3 style="color:#2E86C1; font-family: 'Helvetica';">(4) Core Functions</h2>


In [198]:
def CalculateKey(state: DStarLiteState, s):
    val = min(state.g[s], state.rhs[s])
    return (val + h(state.s_start, s, state.node_positions) + state.k_m, val)

In [199]:
def UpdateVertex(state: DStarLiteState, u):
    in_queue = any(node == u for _, node in state.U)
    if state.g[u] != state.rhs[u] and in_queue:
        U_Update(state.U, u, CalculateKey(state, u))
    elif state.g[u] != state.rhs[u] and not in_queue:
        U_Insert(state.U, u, CalculateKey(state, u))
    elif state.g[u] == state.rhs[u] and in_queue:
        U_Remove(state.U, u)

In [200]:
def Initialize(state: DStarLiteState):
    state.U.clear()
    state.k_m = 0.0
    for s in state.S:
        state.g[s] = float('inf')
        state.rhs[s] = float('inf')

    state.rhs[state.s_goal] = 0.0
    U_Insert(state.U, state.s_goal, CalculateKey(state, state.s_goal))
    UpdateVertex(state, state.s_start)

In [201]:
def ComputeShortestPath(state: DStarLiteState):
    while (KeyLessThan(U_TopKey(state.U), CalculateKey(state, state.s_start))
           or state.rhs[state.s_start] != state.g[state.s_start]):

        if not state.U:
            break

        k_old, u = heapq.heappop(state.U)
        k_new = CalculateKey(state, u)

        if KeyLessThan(k_old, k_new):
            U_Insert(state.U, u, k_new)
            continue

        if state.g[u] > state.rhs[u]:
            state.g[u] = state.rhs[u]
            for s in state.pred[u]:
                if s != state.s_goal:
                    state.rhs[s] = min(state.rhs[s], c(s, u, state.cost_forecast) + state.g[u])
                UpdateVertex(state, s)
        else:
            g_old = state.g[u]
            state.g[u] = float('inf')
            for s in (list(state.pred[u]) + [u]):
                if state.rhs[s] == c(s, u, state.cost_forecast) + g_old:
                    if s != state.s_goal:
                        state.rhs[s] = min(
                            (c(s, sp, state.cost_forecast) + state.g[sp]) for sp in state.succ[s]
                        )
                UpdateVertex(state, s)

In [202]:
def scan_for_edge_changes(state: DStarLiteState, threshold=1e-6, current_node=None, radius=1):
    if current_node is None:
        return []

    visited = {current_node}
    frontier = [current_node]
    changed = []

    for _ in range(radius):
        new_frontier = []
        for u in frontier:
            for v in state.succ[u]:
                forecast_val = state.cost_forecast.get((u, v))
                nominal_val  = state.cost_nominal.get((u, v))
                if nominal_val is None:
                    continue
                if abs(forecast_val - nominal_val) > threshold:
                    changed.append((u, v))
                if v not in visited:
                    visited.add(v)
                    new_frontier.append(v)
        frontier = new_frontier
    return changed

In [203]:
def execute_dstar_from_robust_path(state: DStarLiteState, robust_path: List[int]):
    """Option B – Follow robust path until changes trigger replanning."""
    path_nodes = [state.s_start]
    for next_node in robust_path[1:]:
        state.s_start = next_node
        path_nodes.append(next_node)
        changed_edges = scan_for_edge_changes(state, current_node=state.s_start, radius=state.R_SENSE)
        if changed_edges:
            print(f"Detected {len(changed_edges)} changes near {state.s_start} → replanning.")
            state.k_m += 1
            for (u, v) in changed_edges:
                new_cost = c(u, v, state.cost_nominal)
                state.cost_forecast[(u, v)] = new_cost
                UpdateVertex(state, u)
            ComputeShortestPath(state)
    return path_nodes

In [204]:
def initialize_dstar_with_path(state: DStarLiteState, path_nodes: List[int]):
    """Pre-initialize g/rhs along robust path (warm-start)."""
    for i in range(len(path_nodes) - 1):
        u, v = path_nodes[i], path_nodes[i + 1]
        est_cost = c(u, v, state.cost_forecast)
        state.g[u] = est_cost
        state.rhs[u] = est_cost
    print("Initialized D* Lite state with robust path priors (warm-start).")

In [205]:
def initialize_from_robust_path(state: DStarLiteState, robust_path: List[int]):
    """
    Initialize D* Lite's g and rhs values by processing the robust path
    as if performing a partial backward search (goal -> start).
    """
    state.robust_initialized = True
    # Reset internal values
    state.U.clear()
    for s in state.S:
        state.g[s] = float('inf')
        state.rhs[s] = float('inf')

    # --- Set goal as usual ---
    goal = robust_path[-1]
    state.s_goal = goal
    state.rhs[goal] = 0.0
    U_Insert(state.U, goal, CalculateKey(state, goal))

    # --- Backward relaxations along the robust path ---
    expanded = 0
    for i in range(len(robust_path) - 2, -1, -1):  # from second-to-last → start
        u, v = robust_path[i], robust_path[i + 1]
        cost = c(u, v, state.cost_forecast)
        state.rhs[u] = min(state.rhs[u], cost + state.g[v])
        UpdateVertex(state, u)
        ComputeShortestPath(state)
        expanded += 1

    print(f"✅ Initialized D* Lite along robust path ({expanded} nodes processed).")

In [206]:
def Main(state: DStarLiteState, output_dir=None, visualize=True, nodes_csv=None,
         edges_csv=None, cost_forecast_initial=None, robust_path=None):

    if getattr(state, "robust_initialized", False):
        print("⚙️ Using robust-initialized state – skipping Initialize()")
    else:
        print("🧠 Performing standard D* Lite initialization (A* backward).")
        Initialize(state)
        ComputeShortestPath(state)

    s_last = state.s_start
    path_nodes, step = [state.s_start], 0

    # --- Main forward execution loop ---
    while state.s_start != state.s_goal:
        if state.g[state.s_start] == float('inf'):
            print("No valid path to goal!")
            break
        succ_nodes = state.succ[state.s_start]
        if not succ_nodes:
            print("No successors available from start!")
            break

        state.s_start = min(succ_nodes, key=lambda sp: c(state.s_start, sp, state.cost_forecast) + state.g[sp])
        path_nodes.append(state.s_start)
        step += 1

        changed_edges = scan_for_edge_changes(state, current_node=state.s_start, radius=state.R_SENSE)
        print(f"Step {step}: discovered {len(changed_edges)} changed edges within radius {state.R_SENSE}")

        if changed_edges:
            state.k_m += h(s_last, state.s_start, state.node_positions)
            s_last = state.s_start
            for (u, v) in changed_edges:
                c_old = c(u, v, state.cost_forecast)
                new_cost = c(u, v, state.cost_nominal)
                state.cost_forecast[(u, v)] = new_cost
                if c_old > new_cost:
                    if u != state.s_goal:
                        state.rhs[u] = min(state.rhs[u], new_cost + state.g[v])
                elif state.rhs[u] == c_old + state.g[v]:
                    if u != state.s_goal:
                        state.rhs[u] = min(c(u, sp, state.cost_forecast) + state.g[sp] for sp in state.succ[u])
                UpdateVertex(state, u)
            # --- Diagnostic section ---
            print(f"Step {step}: {len(changed_edges)} changes detected.")
            before_g = state.g[state.s_start]
            ComputeShortestPath(state)
            after_g = state.g[state.s_start]
            print(f"  → g[{state.s_start}] before={before_g:.4f}, after={after_g:.4f}")

    # --- Postprocessing and export ---
    breakdown_df, total_forecast, total_nominal = build_dstar_breakdown(
        state.g, state.rhs, state.U, path_nodes, state.succ, cost_forecast_initial, state.cost_nominal
    )
    breakdown_csv, path_csv = save_dstar_results(output_dir, breakdown_df, path_nodes)
    if visualize:
        plot_dstar_path_overlay(nodes_csv, edges_csv, state.cost_forecast, path_nodes, output_dir,
                                total_forecast, total_nominal, robust_path=robust_path)
    print(f"\nFinal D* Lite path has {len(path_nodes)} nodes.")
    print(f"Total Nominal Cost = {total_nominal:.3f}, Forecast Cost = {total_forecast:.3f}")
    return {
        "path_nodes": path_nodes,
        "breakdown_csv": breakdown_csv,
        "path_csv": path_csv,
        "total_nominal": total_nominal,
        "total_forecast": total_forecast
    }

In [207]:
def run_dstar():
    dir = "/Users/philippstockerl/BachelorThesis/Data/master17/seed_10864_grid100_Stable_ls30.0_var1.0_nug1.0"
    nodes_csv = f"{dir}/nodes.csv"
    edges_csv = f"{dir}/edges.csv"
    params_json = "/Users/philippstockerl/BachelorThesis/Organized/parameters.json"

    # --- Load parameters ---
    if os.path.exists(params_json):
        with open(params_json, "r") as f:
            params = json.load(f)
        print(f"Loaded parameters from {params_json}")
    else:
        raise FileNotFoundError(f"Parameters file not found: {params_json}")

    S, succ, pred, cost_forecast, cost_nominal, node_positions = load_graph_data(nodes_csv, edges_csv)
    cost_forecast_initial = dict(cost_forecast)

    general_params = params.get("general", {})
    dstar_params = params.get("dstar", {})

    print("\nLoaded parameters:")
    print("  Sensing radius:", dstar_params.get("sensing_radius", 1))
    print("  Visualization:", dstar_params.get("visualize", general_params.get("visualize", True)))
    print("  Output dir:", dstar_params.get("output_dir") or general_params.get("output_dir", "dstar_results"))

    robust_path = None
    if dstar_params.get("use_robust_path", False):
        robust_path_file = "/Users/philippstockerl/BachelorThesis/Data/master17/seed_10864_grid100_Stable_ls30.0_var1.0_nug1.0/robust_Gamma_39600.0/robust_path_nodes.csv"
        robust_path = load_robust_path(robust_path_file)

    state = DStarLiteState(
        S=S, succ=succ, pred=pred,
        cost_forecast=cost_forecast,
        cost_nominal=cost_nominal,
        node_positions=node_positions,
        R_SENSE=dstar_params.get("sensing_radius", 1)
    )
    state.s_start = min(S)
    state.s_goal = max(S)

    if robust_path:
        mode = dstar_params.get("robust_mode", "warm_start")
        guided_variant = dstar_params.get("guided_variant", "pure")
        reopt_distance = int(dstar_params.get("reopt_distance", 50))

        if mode == "warm_start":
            initialize_dstar_with_path(state, robust_path)

        elif mode == "robust_init":
            initialize_from_robust_path(state, robust_path)
            # skip normal Initialize() later (we’ll handle that in Step 3)

        elif mode == "guided":
            guided_nodes = execute_dstar_from_robust_path(state, robust_path)
            print(f"Guided D* Lite executed {len(guided_nodes)} steps with online replanning.")

            out_dir = os.path.join(dir, dstar_params.get("output_dir", "dstar_results"))
            os.makedirs(out_dir, exist_ok=True)

            guided_csv = os.path.join(out_dir, "dstar_guided_path_nodes.csv")
            pd.DataFrame({"path_node_id": guided_nodes}).to_csv(guided_csv, index=False)
            print(f"Saved guided path → {guided_csv}")

            plot_dstar_path_overlay(
                nodes_csv, edges_csv, state.cost_forecast, guided_nodes,
                output_dir=out_dir, robust_path=robust_path
            )

            if guided_variant == "pure":
                print("✅ Guided simulation complete (pure mode). No global reoptimization.")
                return  # <— safe exit point instead of killing the kernel

            elif guided_variant == "hybrid":
                if reopt_distance > len(guided_nodes):
                    reopt_distance = len(guided_nodes) - 1
                state.s_start = guided_nodes[-reopt_distance]
                print(f"🔁 Hybrid mode: continuing reoptimization from node "
                      f"{state.s_start} ({reopt_distance} steps before goal).")

    out_dir = os.path.join(dir, dstar_params.get("output_dir") or general_params.get("output_dir", "dstar_results"))
    result = Main(
        state,
        output_dir=out_dir,
        visualize=dstar_params.get("visualize", general_params.get("visualize", True)),
        nodes_csv=nodes_csv,
        edges_csv=edges_csv,
        cost_forecast_initial=cost_forecast_initial,
        robust_path=robust_path
    )

    print("\n=== D* Lite Export Summary ===")
    print(f"Path CSV: {result['path_csv']}")
    print(f"Breakdown CSV: {result['breakdown_csv']}")
    print(f"Total Nominal Cost: {result['total_nominal']:.3f}")
    print(f"Total Forecast Cost: {result['total_forecast']:.3f}")

In [208]:
if __name__ == "__main__":
    run_dstar()

Loaded parameters from /Users/philippstockerl/BachelorThesis/Organized/parameters.json

Loaded parameters:
  Sensing radius: 1
  Visualization: True
  Output dir: dstar_results
Loaded robust path with 199 nodes from /Users/philippstockerl/BachelorThesis/Data/master17/seed_10864_grid100_Stable_ls30.0_var1.0_nug1.0/robust_Gamma_39600.0/robust_path_nodes.csv
✅ Initialized D* Lite along robust path (198 nodes processed).
⚙️ Using robust-initialized state – skipping Initialize()
Step 1: discovered 3 changed edges within radius 1
Step 1: 3 changes detected.
  → g[100] before=89.6393, after=89.9865
Step 2: discovered 3 changed edges within radius 1
Step 2: 3 changes detected.
  → g[200] before=89.0388, after=89.4832
Step 3: discovered 3 changed edges within radius 1
Step 3: 3 changes detected.
  → g[300] before=88.5497, after=88.7858
Step 4: discovered 3 changed edges within radius 1
Step 4: 3 changes detected.
  → g[400] before=87.8596, after=88.0064
Step 5: discovered 3 changed edges within